In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from nbc_analysis.utils.config_utils import get_config
from nbc_analysis.extracts.main import main as run_extracts
from nbc_analysis.utils.debug_utils import runit

In [4]:
runit(get_config)

{'DAYS': ['20190719'],
 'EXTRACT_SPECS': {'ios': {'prefix': 'NBCProd/iOS/NBCUniversal_{day}'},
  'tvOS': {'prefix': 'NBCProd/tvOS/NBC_{day}'}},
 'LIMIT': 3000,
 'RAW_EVENTS_BUCKET': 'nbc-digital-cloned',
 'EVENT_SET_D': '/Users/wmcabee/DATA/NBC2/samples'}

In [9]:
ret = runit(run_extracts)
ret

>> init extract run
>> start extract run, config={'DAYS': ['20190719'], 'EXTRACT_SPECS': {'ios': {'prefix': 'NBCProd/iOS/NBCUniversal_{day}'}, 'tvOS': {'prefix': 'NBCProd/tvOS/NBC_{day}'}}, 'LIMIT': 3000, 'RAW_EVENTS_BUCKET': 'nbc-digital-cloned', 'EVENT_SET_D': '/Users/wmcabee/DATA/NBC2/samples'}
>> start extract,extract=ios,outfile=/Users/wmcabee/DATA/NBC2/samples/ios_20190719.csv
>> end extract,extract=ios,outfile=/Users/wmcabee/DATA/NBC2/samples/ios_20190719.csv,records=234
>> start extract,extract=tvOS,outfile=/Users/wmcabee/DATA/NBC2/samples/tvOS_20190719.csv
>> end extract,extract=tvOS,outfile=/Users/wmcabee/DATA/NBC2/samples/tvOS_20190719.csv,records=609
>> end extract run


In [29]:
import boto3
from toolz import take, first
import re
import pandas as pd
from pathlib import Path
import shutil
import itertools

In [18]:
## Setup work directory

def init_workdir():
    workdir = Path.home() / 'DATA'/ 'NBC'/'samples'
    if workdir.is_dir():
        shutil.rmtree(str(DATA_TOP))
    workdir.mkdir(parents=True)
    return  workdir

In [156]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('nbc-digital-cloned')

In [157]:
print(bucket)

s3.Bucket(name='nbc-digital-cloned')


In [158]:
bucket.name

'nbc-digital-cloned'

In [159]:
bucket.creation_date

datetime.datetime(2019, 9, 7, 14, 39, 51, tzinfo=tzutc())

In [160]:
bucket.get_available_subresources()

['Acl',
 'Cors',
 'Lifecycle',
 'LifecycleConfiguration',
 'Logging',
 'Notification',
 'Object',
 'Policy',
 'RequestPayment',
 'Tagging',
 'Versioning',
 'Website']

In [162]:
def write_sample(extract, spec, day, workdir):
    
    prefix = spec['prefix'].format(day=day)
    print(extract, prefix)
    
    reader =  bucket.objects.filter(Prefix=prefix).all()
    reader = take(spec['limit'], reader)
    reader = ((x.key, x.size) for x in reader)
    df = pd.DataFrame.from_records(reader, columns=['name','size'])
    extract_f = f"{extract}_{day}.csv"
    df['day'] = day
    df['extract'] = extract
    df['extract_f'] = extract_f
    outfile = workdir /  extract_f
    df.to_csv(outfile, index=False)
    df = df.sort_values(['size'])
    print(f"wrote outfile,records={len(df)}")
    return df

In [163]:
workdir = init_workdir()
print(workdir)
reader = iter(EXTRACT_SPECS.items())
reader =( ( extract, spec, day, workdir) for extract, spec in reader for day in DAYS) 
reader = itertools.starmap(write_sample, reader)
#df = first(reader)
for x in reader: pass

/Users/wmcabee/DATA/NBC/samples
android NBCProd/Android/NBC_20190719
wrote outfile,records=1698
android NBCProd/Android/NBC_20190720
wrote outfile,records=1686
android NBCProd/Android/NBC_20190721
wrote outfile,records=1720
android NBCProd/Android/NBC_20190722
wrote outfile,records=1725
roku NBCProd/Roku/NBC_20190719
wrote outfile,records=1697
roku NBCProd/Roku/NBC_20190720
wrote outfile,records=1687
roku NBCProd/Roku/NBC_20190721
wrote outfile,records=1720
roku NBCProd/Roku/NBC_20190722
wrote outfile,records=1725
web NBCProd/Web/NBC_App_20190719
wrote outfile,records=1705
web NBCProd/Web/NBC_App_20190720
wrote outfile,records=1689
web NBCProd/Web/NBC_App_20190721
wrote outfile,records=1721
web NBCProd/Web/NBC_App_20190722
wrote outfile,records=1727
ios NBCProd/iOS/NBCUniversal_20190719
wrote outfile,records=234
ios NBCProd/iOS/NBCUniversal_20190720
wrote outfile,records=209
ios NBCProd/iOS/NBCUniversal_20190721
wrote outfile,records=243
ios NBCProd/iOS/NBCUniversal_20190722
wrote outf

,size
count,1.698000e+03
mean,1.858141e+06
std,8.923466e+05
min,4.880510e+05
25%,1.224130e+06
50%,1.586924e+06
75%,2.173240e+06
max,4.144244e+06


ModuleNotFoundError: No module named 'seaborn'